<a href="https://colab.research.google.com/github/padong2023/006975/blob/master/(5)%EC%9B%B9_%EC%8A%A4%ED%81%AC%EB%9E%98%ED%95%91_%EC%82%AC%EC%9A%A9%ED%95%98%EA%B8%B02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
isbn = '9791190090018'
url = 'https://www.yes24.com/Product/Search?domain=BOOK&query={}'
r=requests.get(url.format(isbn))

In [ ]:
print(r.text)

In [3]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(r.text, 'html.parser')

In [4]:
# 위 검색 결과의 도서 가져오기
prd_link = soup.find('a', attrs={'class':'gd_name'})
print(prd_link)

<a class="gd_name" href="/Product/Goods/74261416" onclick="wiseLogV2('S', '101_005_003_001', ''); setGoodsClickExtraCodeHub('032', '9791190090018', '74261416', '0');">우리가 빛의 속도로 갈 수 없다면</a>


In [5]:
print(prd_link['href'])

/Product/Goods/74261416


In [6]:
url = 'https://www.yes24.com'+prd_link['href']
r=requests.get(url)

In [ ]:
print(r.text)

In [ ]:
soup = BeautifulSoup(r.text, 'html.parser')   # 상세페이지의 파싱
pro_detail = soup.find('div', attrs={'id':'infoset_specific'})
#pro_detail_2 = soupo.find('table', attrs={'class':'tb_nor tb_vertical'})
print(pro_detail)

In [ ]:
prd_tr_list = pro_detail.find_all('tr')
print(prd_tr_list)

In [10]:
# 쪽, 무게, 크기 가져오기
for tr in prd_tr_list:
  if tr.find('th').get_text() == '쪽수, 무게, 크기':
    page_td = tr.find('td').get_text()
    break

In [11]:
print(page_td)

330쪽 | 496g | 130*198*30mm


In [12]:
print(page_td.split()[0])

330쪽


In [ ]:
#### 위의 코드를 사용자 정의 함수로 만들기 ####

def get_page_cnt(isbn):
  url = 'https://www.yes24.com/Product/Search?domain=BOOK&query={}'

  # URL에 isbn을 넣어 HTML 가져옴
  r = requests.get(url.format(isbn))
  soup = BeautifulSoup(r.text, 'html.parser')   #파싱

  # 검색 결과에서 해당 도서 선택
  prd_info = soup.find('a', attrs={'class':'gd_name'})

  # 도서 상세 페이지 가져오기
  url = 'http://www.yes24.com'+prd_info['href']
  r= requests.get(url)
  soup = BeautifulSoup(r.text, 'html.parser')

  # 품목정보 <div>를 선택
  prd_detail = soup.find('div', attrs={'id':'infoset_specific'})

  # 테이블에 있는 <tr> 태그 가져오기
  prd_tr_list = prd_detail.find_all('tr')

  # 쪽수가 들어 있는 <th>를 찾아 <td>에 담긴 값을 반환
  for tr in prd_tr_list:
    if tr.find('th').get_text() == '쪽수, 무게, 크기':
      return tr.find('td').get_text().split()[0]
  return ''

In [ ]:
get_page_cnt(9791190090018)

In [ ]:
# 20대가 가장 좋아하는 도서 찾기 -> API사용하기 소스코드에서 가져오기

import requests
url = "http://data4library.kr/api/loanItemSrch?format=json&startDt=2021-04-01&endDt=2021-04-30&age=20&authKey=85b8722cf86271659f020f7c66208903856a4e95cb5e6630ce8d2cedeac3c15a"

r=requests.get(url)

data=r.json()

books = []
for d in data['response']['docs']:
  books.append(d['doc'])

In [ ]:
import pandas as pd
books_df = pd.DataFrame(books)
books_df

In [ ]:
# joson 파일로 저장하기
books_df.to_json('20s_best_book.json')

In [ ]:
top5_book = books_df.head(5)
top5_book

In [ ]:
# isbn13의 값을 전달하는 사용자 정의 함수 만들기
def get_page_cnt2(row):
  isbn = row['isbn13']
  return get_page_cnt(isbn)

In [ ]:
page_count = top5_book.apply(get_page_cnt2, axis=1)    #axis의 값이 '0'이면 행, '1'이면 열에 적용됨
print(page_count)

In [ ]:
# page_count 시리즈 객체 이름 만들어주기
page_count.name = 'page_count'
print(page_count)

In [ ]:
# 데이터프레임과 시리즈 합치기 : merge()함수
top5_with_page_count = pd.merge(top5_book, page_count,
                                left_index=True, right_index=True)
top5_with_page_count